In [1]:
!pip install torch torchvision transformers accelerate pillow sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 79.3 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found existing installation: nvidia-curand-cu12 10.3.10.19
    Uninstalling nvidia-curand-cu12-

In [12]:
!pip install -q torch torchvision transformers pillow requests gradio accelerate

import os
import torch
from PIL import Image
from transformers import VisionEncoderDecoderModel, ViTImageProcessor, AutoTokenizer
import gradio as gr
import requests
from io import BytesIO
import numpy as np

# Disable tokenizer parallelism warning
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Initialize device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Model configuration
MODEL_NAME = "nlpconnect/vit-gpt2-image-captioning"
MAX_LENGTH = 64
NUM_BEAMS = 4

# Load model components
feature_extractor = ViTImageProcessor.from_pretrained(MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Load model with appropriate precision
model = VisionEncoderDecoderModel.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32
).to(device)

# Configure model
model.config.decoder_start_token_id = tokenizer.cls_token_id
model.config.pad_token_id = tokenizer.pad_token_id
model.config.max_length = MAX_LENGTH
model.config.num_beams = NUM_BEAMS

def process_image(image):
    """Process image and generate caption"""
    try:
        # Convert input to PIL Image
        if isinstance(image, str):
            if image.startswith(("http:", "https:")):
                response = requests.get(image, timeout=10)
                img = Image.open(BytesIO(response.content)).convert("RGB")
            else:
                img = Image.open(image).convert("RGB")
        elif isinstance(image, np.ndarray):
            img = Image.fromarray(image).convert("RGB")
        else:
            img = image.convert("RGB") if hasattr(image, 'convert') else image
        
        # Process image
        pixel_values = feature_extractor(
            images=img, 
            return_tensors="pt"
        ).pixel_values.to(device)
        
        with torch.no_grad():
            output_ids = model.generate(
                pixel_values,
                max_length=model.config.max_length,
                num_beams=model.config.num_beams,
                early_stopping=True
            )
        
        return tokenizer.decode(output_ids[0], skip_special_tokens=True)
    
    except Exception as e:
        return f"Error: {str(e)}"

# Create Gradio interface
with gr.Blocks(title="Image Caption Generator") as demo:
    gr.Markdown(f"""
    # 🖼️ Image Captioning
    **Model:** `{MODEL_NAME}`  
    **Device:** `{device.upper()}`  
    **Max Length:** `{MAX_LENGTH}` | **Beams:** `{NUM_BEAMS}`
    """)
    
    with gr.Row():
        image_input = gr.Image(label="Upload Image", type="pil")
        output_text = gr.Textbox(label="Generated Caption", lines=4)
    
    with gr.Accordion("Advanced Settings", open=False):
        with gr.Row():
            max_length = gr.Slider(16, 128, value=MAX_LENGTH, label="Max Length")
            num_beams = gr.Slider(1, 8, value=NUM_BEAMS, step=1, label="Number of Beams")
    
    def generate_caption(img, ml, beams):
        try:
            pixel_values = feature_extractor(
                images=img,
                return_tensors="pt"
            ).pixel_values.to(device)
            
            with torch.no_grad():
                output_ids = model.generate(
                    pixel_values,
                    max_length=ml,
                    num_beams=beams,
                    early_stopping=True
                )
            
            return tokenizer.decode(output_ids[0], skip_special_tokens=True)
        except Exception as e:
            return f"Generation error: {str(e)}"
    
    image_input.change(
        fn=generate_caption,
        inputs=[image_input, max_length, num_beams],
        outputs=output_text
    )
    
    gr.Examples(
        examples=[
            ["https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/transformers/rabbit.png"],
            ["https://storage.googleapis.com/sfr-vision-language-research/BLIP/demo.jpg"]
        ],
        inputs=[image_input],
        outputs=[output_text],
        fn=lambda img: generate_caption(img, MAX_LENGTH, NUM_BEAMS),
        cache_examples=False
    )

# Launch the interface
try:
    demo.launch(debug=False)
except Exception as e:
    print(f"Interface error: {e}")
    demo.launch(debug=True)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Using device: cuda


Config of the encoder: <class 'transformers.models.vit.modeling_vit.ViTModel'> is overwritten by shared encoder config: ViTConfig {
  "architectures": [
    "ViTModel"
  ],
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": 224,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "pooler_act": "tanh",
  "pooler_output_size": 768,
  "qkv_bias": true,
  "torch_dtype": "float16",
  "transformers_version": "4.51.3"
}

Config of the decoder: <class 'transformers.models.gpt2.modeling_gpt2.GPT2LMHeadModel'> is overwritten by shared decoder config: GPT2Config {
  "activation_function": "gelu_new",
  "add_cross_attention": true,
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "decoder_start_to

* Running on local URL:  http://127.0.0.1:7864
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://09cf85aa05a949d1cd.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.11/dist-packages/transformers/generation/utils.py:1667: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed in v5. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
You may ignore this warning if your `pad_token_id` (50256) is identical to the `bos_token_id` (50256), `eos_token_id` (50256), or the `sep_token_id` (None)